In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/03 14:20:00 WARN Utils: Your hostname, coy-pc resolves to a loopback address: 127.0.1.1; using 172.19.138.153 instead (on interface eth0)
24/03/03 14:20:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/03 14:20:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/03 14:20:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [5]:
df_green.createOrReplaceTempView('green')

In [13]:
df_green_revenue = spark.sql("""

    SELECT 
    -- Reveneue grouping 
    date_trunc('hour', lpep_pickup_datetime) as hour, 
    pulocationid as zone,

    -- Revenue calculation 
    SUM(total_amount) as amount,
    COUNT(1) AS number_records

    FROM
        green
    WHERE
        lpep_pickup_datetime >= '2020-01-01'
    GROUP BY
        1,2
    -- ORDER BY
    --    1,2

""")

In [14]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-09 17:00:00| 225|             82.46|             5|
|2020-01-02 21:00:00|  74| 506.4200000000001|            38|
|2020-01-27 13:00:00|  24|            203.57|             7|
|2020-01-06 16:00:00|  43|343.68000000000006|            21|
|2020-01-05 08:00:00|  55|132.51999999999998|             4|
|2020-01-20 15:00:00|  82|            380.22|            27|
|2020-01-28 22:00:00| 255|217.10999999999999|            13|
|2020-01-27 09:00:00| 177|             102.8|             3|
|2020-01-26 20:00:00|  35|             48.61|             2|
|2020-01-29 08:00:00| 223|            119.93|             8|
|2020-01-16 10:00:00|  82|273.62000000000006|            18|
|2020-01-24 18:00:00| 210|            124.58|             6|
|2020-01-02 23:00:00|  66|            258.49|            11|
|2020-01-14 14:00:00| 22

In [15]:
df_green_revenue.write.parquet('data/report/revenue/green', mode="overwrite")

24/03/03 14:29:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/03 14:29:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/03 14:29:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/03 14:29:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/03/03 14:29:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/03/03 14:29:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
24/03/03 14:29:50 WARN MemoryManager: Total allocation exceeds 95.

In [16]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.createOrReplaceTempView('yellow')

In [18]:
df_yellow_revenue = spark.sql("""

    SELECT 
    -- Reveneue grouping 
    date_trunc('hour', tpep_pickup_datetime) as hour, 
    pulocationid as zone,

    -- Revenue calculation 
    SUM(total_amount) as amount,
    COUNT(1) AS number_records

    FROM
        yellow
    WHERE
        tpep_pickup_datetime >= '2020-01-01'
    GROUP BY
        1,2
    -- ORDER BY
    --    1,2

""")

In [19]:
df_yellow_revenue.write.parquet('data/report/revenue/yellow', mode='overwrite')

24/03/03 14:39:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/03 14:39:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/03 14:39:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/03 14:39:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/03/03 14:39:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/03/03 14:39:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
24/03/03 14:39:29 WARN MemoryManager: Total allocation exceeds 95.

In [22]:
# rename columns so that we can see them nicely in joined table
df_green_revenue_renamed = df_green_revenue \
        .withColumnRenamed('amount', 'green_amount') \
        .withColumnRenamed('number_records', 'green_number_records') 

df_yellow_revenue_renamed = df_yellow_revenue \
        .withColumnRenamed('amount', 'yellow_amount') \
        .withColumnRenamed('number_records', 'yellow_number_records') 

In [23]:
df_join = df_green_revenue_renamed.join(
    df_yellow_revenue_renamed, 
    on=['hour', 'zone'],
    how='outer'
)

In [24]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  65|199.48999999999998|                  10|            409.35|                   19|
|2020-01-01 00:00:00|  68|              null|                null| 7825.069999999994|                  396|
|2020-01-01 00:00:00|  88|              null|                null| 823.8000000000001|                   36|
|2020-01-01 00:00:00|  93|              null|                null|            210.28|                    3|
|2020-01-01 00:00:00| 106|             10.56|                   1|              null|                 null|
|2020-01-01 00:00:00| 194|              null|                null|            148.71|                    2|
|2020-01-01 00:00:00| 248|  

In [25]:
df_join.write.parquet('data/report/revenue/total')

24/03/03 15:02:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/03 15:02:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/03 15:02:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/03 15:02:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/03/03 15:02:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/03/03 15:02:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
24/03/03 15:02:46 WARN MemoryManager: Total allocation exceeds 95.

24/03/03 15:02:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 50.67% for 15 writers
24/03/03 15:02:47 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 54.29% for 14 writers
24/03/03 15:02:47 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
24/03/03 15:02:47 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/03/03 15:02:47 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/03/03 15:02:47 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/03 15:02:47 WARN MemoryManager: Total allocation exceeds 9

In [ ]:
# doing it all again but this time from a materialized (parquet) data (di na irerepeat steps)

In [26]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [27]:

# rename columns so that we can see them nicely in joined table
df_green_revenue_renamed = df_green_revenue \
        .withColumnRenamed('amount', 'green_amount') \
        .withColumnRenamed('number_records', 'green_number_records') 

df_yellow_revenue_renamed = df_yellow_revenue \
        .withColumnRenamed('amount', 'yellow_amount') \
        .withColumnRenamed('number_records', 'yellow_number_records') 

In [28]:
df_join = df_green_revenue_renamed.join(
    df_yellow_revenue_renamed, 
    on=['hour', 'zone'],
    how='outer'
)

In [29]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  65|199.48999999999998|                  10|            409.35|                   19|
|2020-01-01 00:00:00|  68|              null|                null| 7825.069999999994|                  396|
|2020-01-01 00:00:00|  88|              null|                null| 823.8000000000001|                   36|
|2020-01-01 00:00:00|  93|              null|                null|            210.28|                    3|
|2020-01-01 00:00:00| 106|             10.56|                   1|              null|                 null|
|2020-01-01 00:00:00| 194|              null|                null|            148.71|                    2|
|2020-01-01 00:00:00| 248|  

In [30]:
# plot twist:
# so in the Spark Master console (localhost:4040)
# the job plan does not differ vs on previous join haha

In [ ]:
# joining on large and small tables

In [31]:
df_join = spark.read.parquet('data/report/revenue/total')

In [32]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  17|            195.03|                   9|220.20999999999998|                    8|
|2020-01-01 00:00:00|  77| 75.99000000000001|                   2|            134.24|                    3|
|2020-01-01 00:00:00|  95|202.20000000000002|                  15|               6.3|                    1|
|2020-01-01 00:00:00| 126|              null|                null|             170.6|                    2|
|2020-01-01 00:00:00| 133|              null|                null|              27.3|                    1|
|2020-01-01 00:00:00| 161|              null|                null|           9410.21|                  488|
|2020-01-01 00:00:00| 228|  

In [34]:
df_zones = spark.read.parquet('zones/')

In [35]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [36]:
df_result = df_join.join(
    df_zones,
    df_join.zone == df_zones.LocationID,
)

In [38]:
df_result.drop('locationid').show()

+-------------------+----+------------------+--------------------+------------------+---------------------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+---------+--------------------+------------+
|2020-01-01 00:00:00|  17|            195.03|                   9|220.20999999999998|                    8| Brooklyn|             Bedford|   Boro Zone|
|2020-01-01 00:00:00|  77| 75.99000000000001|                   2|            134.24|                    3| Brooklyn|East New York/Pen...|   Boro Zone|
|2020-01-01 00:00:00|  95|202.20000000000002|                  15|               6.3|                    1|   Queens|        Forest Hills|   Boro Zone|
|2020-01-01 00:00:00| 126|              null|                null|             170.6|   

In [41]:
df_result.drop('locationid', 'zone').write.parquet('tmp/revenue-zones')

24/03/03 15:26:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/03 15:26:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/03 15:26:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/03 15:26:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/03/03 15:26:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/03/03 15:26:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
24/03/03 15:26:57 WARN MemoryManager: Total allocation exceeds 95.

24/03/03 15:26:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 50.67% for 15 writers
24/03/03 15:26:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 54.29% for 14 writers
24/03/03 15:26:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
24/03/03 15:26:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/03/03 15:26:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/03/03 15:26:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/03/03 15:26:57 WARN MemoryManager: Total allocation exceeds 9